In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import theano
import theano.tensor as tt
from theano import shared

In [2]:
constit_data_df = pd.read_csv("csvs/MRP/constit_Data.csv")
ge2019_results_df = pd.read_csv("csvs/ge2019.csv")
ge2017_results_df = pd.read_csv("csvs/ge2017.csv")

ge2017_results_df.rename({"first_party": "2017_winner", "region_name": "region_name_2017", "electorate": "2017_electorate", "constituency_name": "2017_constituency_name"}, axis=1, inplace=True)

# Merge the tables together so that we can get the electorate
constits_df = pd.merge(constit_data_df, ge2019_results_df, left_on="ID", right_on="ons_id")
constits_df = pd.merge(constits_df, ge2017_results_df, left_on="ID", right_on="ons_id")
constits_df = constits_df.filter(["ID", "constituency_name", "electorate", "region_name_2017", "PopDensity", "%Unemployment", "%Heavy Industry & Manufacturing", "%Level4+", "%White", '%LeaveVote', "2017_winner"])

constits_df.head()

,ID,constituency_name,electorate,region_name_2017,PopDensity,%Unemployment,%Heavy Industry & Manufacturing,%Level4+,%White,%LeaveVote,2017_winner
0,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.26700,0.855334,0.579,Con
1,E14000531,Aldridge-Brownhills,60138,West Midlands,1759.478824,0.032333,0.258,0.21625,0.931288,0.678,Con
2,E14000532,Altrincham and Sale West,73107,North West,2000.175498,0.019401,0.148,0.41225,0.904339,0.386,Con
3,E14000533,Amber Valley,69976,East Midlands,727.067108,0.032759,0.315,0.19725,0.983888,0.653,Con
4,E14000534,Arundel and South Downs,81726,South East,157.432894,0.013748,0.167,0.33975,0.977485,0.497,Con


In [3]:
# Creat the mrp data by merging the survey data with constituency level data
survey_df = pd.read_csv("csvs/MRP/final_dataset.csv")
survey_df.drop(["constituency_name"], axis=1, inplace=True)
mrp_df = pd.merge(constits_df, survey_df, how='inner', left_on="ID", right_on="constituency_id")
mrp_df.drop(["constituency_id"], axis=1, inplace=True)
mrp_df.head()

,ID,constituency_name,electorate,region_name_2017,PopDensity,%Unemployment,%Heavy Industry & Manufacturing,%Level4+,%White,%LeaveVote,2017_winner,housing_ownership,age_group,sex,education_level,social_grade,voting_intention
0,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.267,0.855334,0.579,Con,Owns,45-59,2.0,Level 2,DE,Conservative
1,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.267,0.855334,0.579,Con,Owns,30-44,2.0,Level 4/5,C2,Conservative
2,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.267,0.855334,0.579,Con,Owns,45-59,2.0,Level 3,C2,Conservative
3,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.267,0.855334,0.579,Con,Rents,45-59,2.0,Level 1,DE,Conservative
4,E14000530,Aldershot,72617,South East,1992.396518,0.017457,0.158,0.267,0.855334,0.579,Con,Owns,45-59,1.0,Other,C2,Liberal Democrat


In [15]:
# Filter out all instances not from Scottish voters
scotland_mrp_df = mrp_df[mrp_df["ID"].str.contains("S")]
scotland_mrp_df.head()

no_constits_to_leave = 15
constits_to_keep = scotland_mrp_df["ID"].value_counts().index[:-no_constits_to_leave]
scotland_mrp_df = scotland_mrp_df[scotland_mrp_df["ID"].isin(constits_to_keep)]
scotland_mrp_df.shape

(2892, 17)

In [16]:
sex_mapping = {
    1: 0,
    2: 1
}

age_mapping = {
    "16-19": 0,
    "20-24": 1,
    "25-29": 2,
    "30-44": 3,
    "45-59": 4,
    "60-64": 5,
    "65-74": 6,
    "75+": 7
}

home_mapping = {
    "Rents": 0,
    "Owns": 1
}

education_mapping = {
    "No qualifications": 0,
    "Other": 1,
    "Level 1": 2,
    "Level 2": 3,
    "Level 3": 4,
    "Level 4/5": 5
}

social_grade_mapping = {
    "DE": 0,
    "C2": 1,
    "C1": 2,
    "AB": 3
}

scotland_vote_mapping = {}
party_counter = 0
for party in scotland_mrp_df["voting_intention"].value_counts().index:
    scotland_vote_mapping[party] = party_counter
    party_counter = party_counter + 1

# level 1 independent variables
sex = scotland_mrp_df["sex"].map(sex_mapping).values
n_sex = len(np.unique(sex))
age = scotland_mrp_df["age_group"].map(age_mapping).values
n_age = len(np.unique(age))
home = scotland_mrp_df["housing_ownership"].map(home_mapping).values
n_home = len(np.unique(home))
edu = scotland_mrp_df["education_level"].map(education_mapping).values
n_edu = len(np.unique(edu))
social_grade = scotland_mrp_df["social_grade"].map(social_grade_mapping).values
n_social_grade = len(np.unique(social_grade))
vote = scotland_mrp_df["voting_intention"].map(scotland_vote_mapping).values
n_choices = len(np.unique(vote))

# level 2 independent variables
# Constituency
unique_con_ids = np.unique(scotland_mrp_df["ID"].values)
counter = 0
id_to_counter = {}
for i in unique_con_ids:
    id_to_counter[i] = counter
    counter = counter + 1
constituency = scotland_mrp_df["ID"].map(id_to_counter).values
n_constituencies = len(unique_con_ids)

# Incumbency
unique_2017_winners = np.unique(scotland_mrp_df["2017_winner"].values)
counter = 0
incumbent_to_counter = {}
for i in unique_2017_winners:
    incumbent_to_counter[i] = counter
    counter = counter + 1
incumbent = scotland_mrp_df["2017_winner"].map(incumbent_to_counter).values
n_incumbents = len(unique_2017_winners)

# Region (Not valid for anything put just England)
#unique_regions = np.unique(scotland_mrp_df["region_name_2017"].values)
#counter = 0
#region_to_counter = {}
#for i in unique_regions:
#    region_to_counter[i] = counter
#    counter = counter + 1
#region = scotland_mrp_df["region_name_2017"].map(region_to_counter).values
#n_regions = len(unique_2017_winners)

l2_data = scotland_mrp_df[["%Heavy Industry & Manufacturing", "%LeaveVote", "%Level4+", "%Unemployment"]].values
n_l2_variables = l2_data.shape[1]

In [17]:
n_parties = n_choices

industry = scotland_mrp_df[["%Heavy Industry & Manufacturing",]].values
leave = scotland_mrp_df[["%LeaveVote"]].values
l4_plus = scotland_mrp_df[["%Level4+"]].values
unemployment = scotland_mrp_df[["%Unemployment"]].values

In [18]:
with pm.Model() as individual_model:
    # Level 2 =======================================================
    # Constituency level
    constit_no = pm.Normal("constit_no", 0., 1., shape=(n_constituencies, n_choices))
    #constit_region = pm.Normal("constit_region", 0., 1., shape=(n_regions, n_choices))
    constit_incumbent = pm.Normal("constit_incumbent", 0., 1., shape=(n_incumbents, n_parties))
    constit_industry = pm.Normal("constit_industry", 0., 1., shape=n_choices)
    constit_leave = pm.Normal("constit_leave", 0., 1., shape=n_choices)
    constit_l4_plus = pm.Normal("constit_l4_plus", 0., 1., shape=n_choices)
    constit_unemployment = pm.Normal("constit_unemployment", 0., 1., shape=n_choices)
    
    # Level 1 =======================================================
    party_intercept = pm.Normal('party_intercept', 0., 1., shape=n_parties)
    
    # Individual level
    individual_sex = pm.Normal('individual_sex', 0., 1., shape=(n_sex, n_parties))
    individual_age = pm.Normal('individual_age', 0., 1., shape=(n_age, n_parties))
    individual_home = pm.Normal('individual_home', 0., 1., shape=(n_home, n_parties))
    individual_edu = pm.Normal('individual_edu', 0., 1., shape=(n_edu, n_parties))
    individual_soc_grade = pm.Normal('individual_soc_grade', 0., 1., shape=(n_social_grade, n_parties))
    
    #equations = party_intercept + individual_sex[sex] + individual_age[age] + individual_home[home] + individual_edu[edu] + individual_soc_grade[social_grade] + l2_equation
    equations = party_intercept + individual_sex[sex] + individual_age[age] + individual_home[home] + individual_edu[edu] + individual_soc_grade[social_grade] + constit_industry*industry + constit_l4_plus*l4_plus + constit_leave*leave + constit_industry*industry + constit_no[constituency] + constit_incumbent[incumbent]
    
    probabilities = pm.Deterministic('probabilities', tt.nnet.softmax(equations))
    
    y = pm.Categorical("y", p=probabilities, observed=vote)
    
    trace = pm.sample()

/home/graeme/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [individual_soc_grade, individual_edu, individual_home, individual_age, individual_sex, party_intercept, constit_unemployment, constit_l4_plus, constit_leave, constit_industry, constit_incumbent, constit_no]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [07:36<00:00,  8.75draws/s]
/home/graeme/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr

In [19]:
post_strat_frame = pd.read_csv("csvs/MRP/poststratification_frame.csv")

post_strat_frame.rename(columns={"hrsocgrd": "social_grade", "housing": "housing_ownership", "education": "education_level", "age0": "age_group", "GSSCode": "constituency_id"}, inplace=True)

post_strat_frame["social_grade"] = post_strat_frame["social_grade"].map(social_grade_mapping)
post_strat_frame["housing_ownership"] = post_strat_frame["housing_ownership"].map(home_mapping)
post_strat_frame["education_level"] = post_strat_frame["education_level"].map(education_mapping)
post_strat_frame["age_group"] = post_strat_frame["age_group"].map({
    "16-19": 0,
    "20-24": 1,
    "25-29": 2,
    "30-44": 3,
    "45-59": 4,
    "60-64": 5,
    "65-74": 6,
    "75": 7
})
post_strat_frame["sex"] = post_strat_frame["sex"].map({
    "Male": 0,
    "Female": 1
})

post_strat_frame.head()

,constituency_id,sex,age_group,housing_ownership,social_grade,education_level,weight
0,E14000530,1,0,1,3,2,0.000097
1,E14000530,1,0,1,3,3,0.000584
2,E14000530,1,0,1,3,4,0.000210
3,E14000530,1,0,1,3,5,0.000012
4,E14000530,1,0,1,3,0,0.000059


In [20]:
scotland_post_strat_frame = post_strat_frame[post_strat_frame["constituency_id"].str.contains("S")]
scotland_post_strat_frame = scotland_post_strat_frame[scotland_post_strat_frame["constituency_id"].isin(constits_to_keep)]

id_to_result = {}
parties = scotland_mrp_df["voting_intention"].value_counts().index

total_wins_per_party = scotland_vote_mapping.copy()
for party in parties:
    total_wins_per_party[party] = 0

total_votes = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for current_constit in range(0, n_constituencies):
    total_constit_votes = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    constit_id = scotland_post_strat_frame.iloc[current_constit*768, 0]
    constit_data = constits_df.loc[constits_df["ID"] == constit_id]
    constit_no = id_to_counter[constit_id]
    #constit_region_no = region_to_counter[constit_data["region_name_2017"].values[0]]
    constit_incumbent_no = incumbent_to_counter[constit_data["2017_winner"].values[0]]
    constit_electorate = constit_data["electorate"].values[0]
    constit_industry = constit_data["%Heavy Industry & Manufacturing"].values[0]
    constit_leave = constit_data["%LeaveVote"].values[0]
    constit_level4 = constit_data['%Level4+'].values[0]
    constit_unemploy = constit_data["%Unemployment"].values[0]
    industry_coeff = trace['constit_industry'].mean(axis=0)
    leave_coeff = trace['constit_leave'].mean(axis=0)
    level4_coeff = trace['constit_l4_plus'].mean(axis=0)
    unemployment_coeff = trace['constit_unemployment'].mean(axis=0)
    constit_term = trace["constit_no"][constit_no].mean(axis=0)
    #region_term = trace["l2_region"][constit_region_no].mean(axis=0)
    incumbent_term = trace["constit_incumbent"][constit_incumbent_no].mean(axis=0)
    intercept = trace['party_intercept'].mean(axis=0)
    for current_cell in  range(0, 768):
        constit_sex = trace["individual_sex"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 1]].mean(axis=0)
        constit_age = trace["individual_age"][int(scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 2])].mean(axis=0)
        constit_home = trace["individual_home"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 3]].mean(axis=0)
        constit_soc_g = trace["individual_soc_grade"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 4]].mean(axis=0)
        constit_edu = trace["individual_edu"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 5]].mean(axis=0)
        
        l2Prediction = incumbent_term + industry_coeff*constit_industry + level4_coeff*constit_level4 + unemployment_coeff*constit_unemploy + leave_coeff*constit_leave + constit_term
        predictions = intercept + constit_sex + constit_age + constit_home + constit_soc_g + constit_edu + l2Prediction
        output = np.exp(predictions).T / np.sum(np.exp(predictions))
        total_constit_votes = total_constit_votes + (output * constit_electorate * scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 6])
    
    winning_party = parties[np.argmax(total_constit_votes)]
    total_wins_per_party[winning_party] = total_wins_per_party[winning_party] + 1
    
    print(constit_id)
    i = 0
    for party in parties:
        print('{:>20}'.format(str(party) + ": ") + str(total_constit_votes[i]))
        i = i + 1
    print("===========================================================================")
    
    total_votes = total_votes + total_constit_votes
    
    
print("")
print("")
print("Final results:")
print(total_wins_per_party)
print("="*30)
total_no_votes = sum(total_votes)
i = 0
for party in parties:
    print('{:>20}'.format(str(party) + ": ") + str(total_votes[i]/total_no_votes))
    i = i + 1

S14000001
               SNP: 25565.463331617757
      Conservative: 9273.120232802012
            Labour: 14215.353892134743
  Liberal Democrat: 7145.514876637822
    Would not vote: 5133.20699881889
      Brexit Party: 399.99992080687304
       Green Party: 586.5394485188257
       Independent: 145.4921887648322
              UKIP: 24.317383162102818
S14000002
               SNP: 22374.162722475336
      Conservative: 14213.537120900906
            Labour: 13546.468863817965
  Liberal Democrat: 9297.549026366632
    Would not vote: 4333.480956248249
      Brexit Party: 287.2911270811818
       Green Party: 1474.616329330524
       Independent: 154.92174459280636
              UKIP: 36.936497208009136
S14000005
               SNP: 22164.08050980318
      Conservative: 12711.4112662193
            Labour: 16834.559968134858
  Liberal Democrat: 7518.488426705961
    Would not vote: 5295.904413815504
      Brexit Party: 358.11739051848105
       Green Party: 1388.2430936406192
       Ind

In [ ]:
print(constits_df[constits_df["constituency_name"] == "Moray"]["ID"].values[0])

# OLD STUFF

In [ ]:
with pm.Model() as individual_model:

    # Level 2 =======================================================
    #l2_constit = pm.Normal("l2_constit", 0., 1., shape=(n_constituencies, n_choices))
    #l2_region = pm.Normal("l2_region", 0., 1., shape=(n_regions, n_choices))
    l2_incumbent = pm.Normal("l2_incumbent", 0., 1., shape=(n_incumbents, n_choices))
    l2_coeffs = pm.Normal('l2_coeffs', 0., 1., shape=(n_l2_variables, n_choices))
    l2_data = pm.Data('l2_data', l2_data)
    l2_equation = pm.Deterministic("l2_equation", l2_incumbent[incumbent] + tt.dot(l2_data, l2_coeffs)) # l2_constit[constituency] + l2_region[region] 
    
    # Level 1 =======================================================
    l1_intercept = pm.Normal('l1_intercept', 0., 1., shape=n_choices)
    
    l1_sex_rand = pm.Normal('l1_sex_rand', 0., 1., shape=(n_sex, n_choices))
    l1_age_rand = pm.Normal('l1_age_rand', 0., 1., shape=(n_age, n_choices))
    l1_home_rand = pm.Normal('l1_home_rand', 0., 1., shape=(n_home, n_choices))
    l1_edu_rand = pm.Normal('l1_edu_rand', 0., 1., shape=(n_edu, n_choices))
    l1_soc_grad_rand = pm.Normal('l1_soc_grad_rand', 0., 1., shape=(n_social_grade, n_choices))
    
    equations = l1_intercept + l1_sex_rand[sex] + l1_age_rand[age] + l1_home_rand[home] + l1_edu_rand[edu] + l1_soc_grad_rand[social_grade] + l2_equation
    
    probabilities = pm.Deterministic('probabilities', tt.nnet.softmax(equations))
    
    y = pm.Categorical("y", p=probabilities, observed=vote)
    
    trace = pm.sample()

In [ ]:
scotland_post_strat_frame = post_strat_frame[post_strat_frame["constituency_id"].str.contains("S")]
#scotland_post_strat_frame = scotland_post_strat_frame[scotland_post_strat_frame["constituency_id"].isin(constits_to_keep)]

id_to_result = {}
parties = scotland_mrp_df["voting_intention"].value_counts().index

total_wins_per_party = scotland_vote_mapping.copy()
for party in parties:
    total_wins_per_party[party] = 0

for current_constit in range(0, n_constituencies):
    total_constit_votes = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    constit_id = scotland_post_strat_frame.iloc[current_constit*768, 0]
    constit_data = constits_df.loc[constits_df["ID"] == constit_id]
    constit_no = id_to_counter[constit_id]
    #constit_region_no = region_to_counter[constit_data["region_name_2017"].values[0]]
    constit_incumbent_no = incumbent_to_counter[constit_data["2017_winner"].values[0]]
    constit_electorate = constit_data["electorate"].values[0]
    constit_industry = constit_data["%Heavy Industry & Manufacturing"].values[0]
    constit_leave = constit_data["%LeaveVote"].values[0]
    constit_level4 = constit_data['%Level4+'].values[0]
    constit_unemploy = constit_data["%Unemployment"].values[0]
    industry_coeff = trace['l2_coeffs'][0].mean(axis=0)
    leave_coeff = trace['l2_coeffs'][1].mean(axis=0)
    level4_coeff = trace['l2_coeffs'][2].mean(axis=0)
    unemployment_coeff = trace['l2_coeffs'][3].mean(axis=0)
    #constit_term = trace["l2_constit"][constit_no].mean(axis=0)
    #region_term = trace["l2_region"][constit_region_no].mean(axis=0)
    incumbent_term = trace["l2_incumbent"][constit_incumbent_no].mean(axis=0)
    intercept = trace['l1_intercept'].mean(axis=0)
    for current_cell in  range(0, 768):
        constit_sex = trace["l1_sex_rand"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 1]].mean(axis=0)
        constit_age = trace["l1_age_rand"][int(scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 2])].mean(axis=0)
        constit_home = trace["l1_home_rand"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 3]].mean(axis=0)
        constit_soc_g = trace["l1_soc_grad_rand"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 4]].mean(axis=0)
        constit_edu = trace["l1_edu_rand"][scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 5]].mean(axis=0)
        
        l2Prediction = incumbent_term + industry_coeff*constit_industry + leave_coeff*constit_leave + level4_coeff*constit_level4 + unemployment_coeff*constit_unemploy # constit_term + region_term
        predictions = intercept + constit_sex + constit_age + constit_home + constit_soc_g + constit_edu + l2Prediction
        output = np.exp(predictions).T / np.sum(np.exp(predictions))
        total_constit_votes = total_constit_votes + (output * constit_electorate * scotland_post_strat_frame.iloc[int(current_constit*768+current_cell), 6])
    
    winning_party = parties[np.argmax(total_constit_votes)]
    total_wins_per_party[winning_party] = total_wins_per_party[winning_party] + 1
    
    print(constit_id)
    i = 0
    for party in parties:
        print('{:>20}'.format(str(party) + ": ") + str(total_constit_votes[i]))
        i = i + 1
    print("===========================================================================")
    
    
print("")
print("")
print("Final results:")
print(total_wins_per_party)